In [ ]:
####################################
########  必要なライブラリ群 ##########
####################################
# 標準ライブラリ
from typing import List, Dict, Any
import pandas as pd
from datasets import Dataset
from io import BytesIO
import time
from datetime import datetime

# Oracle Database
import oracledb

# OCI
import oci

# LangChain
from langchain_community.embeddings import OCIGenAIEmbeddings
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

# OCI GenAI Chat Model
from oci.generative_ai_inference.models import (
    CohereChatRequest,
    OnDemandServingMode,
    ChatDetails
)

# Reranker
from sentence_transformers import CrossEncoder
import torch

# RAGAS
from ragas import evaluate
from ragas.metrics import Faithfulness, AnswerCorrectness, ContextPrecision, ContextRecall
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
from langchain_core.outputs import LLMResult

# 設定読み込み（config_loader.pyと同じディレクトリに配置）
from config_loader import (
    load_config,
    get_db_connection_params,
    get_oci_config,
    get_genai_config,
    get_object_storage_config
)

In [ ]:
################
#  定数及び変数  # 
################

# .envファイルから環境変数を読み込み
load_config()

# 表名
source_documents_table = "source_documents"
chunks_table = "chunks"

# Oracle Database接続
db_params = get_db_connection_params()
connection = oracledb.connect(**db_params)
print(connection)

# OCI設定
config = get_oci_config()
genai_config = get_genai_config()
os_config = get_object_storage_config()

# 設定値を変数に展開
region = config.get('region', 'ap-osaka-1')
compartment_id = genai_config['compartment_id']

# FAQファイル用のObject Storage
bucket_name = "faq"
object_name = "faq.xlsx"

# LLM
embedding_model = "cohere.embed-v4.0"
chat_model = "cohere.command-a-03-2025"
rerank_model = "hotchpotch/japanese-reranker-base-v2"
service_endpoint = f"https://inference.generativeai.{region}.oci.oraclecloud.com"

print(f"\n✓ 設定読み込み完了")
print(f"  - Region: {region}")
print(f"  - Embedding Model: {embedding_model}")
print(f"  - Chat Model: {chat_model}")
print(f"  - Rerank Model: {rerank_model}")
print(f"  - FAQ Bucket: {bucket_name}")

In [ ]:
######################################
# Object StorageからExcelファイルを取得 #
######################################
def load_excel_from_object_storage(config, bucket_name, object_name="faq.xlsx", sheet_name=0):
    """
    OCI Object StorageからExcelファイルを読み込み、DataFrameを返す
    
    Args:
        config (dict): OCI設定情報
        bucket_name (str): Object Storageのバケット名
        object_name (str): Object Storageのオブジェクト名（デフォルト: "faq.xlsx"）
        sheet_name (str or int): 読み込むシート名またはインデックス（デフォルト: 0）
    
    Returns:
        df (DataFrame): データを含むDataFrame
    """
    object_storage_client = oci.object_storage.ObjectStorageClient(config)    
    namespace = object_storage_client.get_namespace().data #type: ignore
    
    # Object StorageからExcelファイルを取得
    get_object_response = object_storage_client.get_object(namespace, bucket_name, object_name)
    
    # バイナリデータをExcelとして読み込み
    excel_data = BytesIO(get_object_response.data.content) #type: ignore
    df = pd.read_excel(excel_data, sheet_name=sheet_name)
    
    # 必要な列が存在するか確認（FAQファイルまたはResultsシートの場合のみ）
    if sheet_name == 0 or sheet_name == 'Results':
        required_columns = ['id', 'question', 'ground_truth', 'filter']
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"必要な列が不足しています: {missing_columns}")
    
    print("\n✓ データの読み込みが完了しました")
    
    return df

In [ ]:
##############################################
# 生成された回答をExcelとしてObject Storageに保存 #
##############################################

def save_to_object_storage(df, config, bucket_name, output_filename, metadata_df=None):
    """
    DataFrameをExcelファイルとしてOCI Object Storageに保存
    
    Args:
        df (DataFrame): 保存するDataFrame
        config (dict): OCI設定情報
        bucket_name (str): Object Storageのバケット名
        output_filename (str): 保存するファイル名（例: "rag_results.xlsx"）
        metadata_df (DataFrame, optional): メタデータ用のDataFrame
    
    Returns:
        str: アップロードしたファイルのURL情報
    """
    excel_buffer = BytesIO()
    
    # ExcelWriterで複数シート対応
    with pd.ExcelWriter(excel_buffer, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='Results', index=False)
        if metadata_df is not None:
            metadata_df.to_excel(writer, sheet_name='Settings', index=False)
    
    excel_buffer.seek(0)
    
    # Object Storageクライアントの作成
    object_storage_client = oci.object_storage.ObjectStorageClient(config)
    namespace = object_storage_client.get_namespace().data #type: ignore
    
    try:
        # Object Storageにアップロード
        object_storage_client.put_object(
            namespace_name=namespace,
            bucket_name=bucket_name,
            object_name=output_filename,
            put_object_body=excel_buffer.getvalue()
        )
        
        print(f"✓ Object Storageへのアップロード成功")
        print(f"  バケット: {bucket_name}")
        print(f"  ファイル名: {output_filename}")
        print(f"  行数: {len(df)}")
        
        return f"Successfully uploaded to {bucket_name}/{output_filename}"
        
    except Exception as e:
        print(f"✗ アップロードエラー: {e}")
        raise

In [ ]:
################
# Embeddingする #
################

generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=service_endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

embeddings = OCIGenAIEmbeddings(
  model_id         = embedding_model,
  service_endpoint = service_endpoint,
  truncate         = "NONE",
  compartment_id   = compartment_id,
  auth_type        = "API_KEY",
  client=generative_ai_inference_client
)

def embed_text(text):
    """
    単一の文字列をEmbeddingしてベクトルデータを返す
    
    Args:
        text (str): Embedding対象の文字列
        
    Returns:
        embedding (str_vector): Embeddingベクトル文字列（浮動小数点数のリスト文字列）
    """
    
    # Embeddingの実行 (pythonのリスト形式): [0.1, 0.2, 0.3,...]
    embedding_ = embeddings.embed_query(text)

    # データベースへのロード用に型変換: [0.1, 0.2, 0.3,...] → "[0.1, 0.2, 0.3,...]"
    embedding = str(embedding_)
        
    return embedding

In [ ]:
##############
# ベクトル検索 #
##############

def vector_search(
    query: str,
    top_k: int = 5,
    filtering: str | None = None
) -> List[Dict[str, Any]]:
    """
    ベクトル類似度検索を実行（チャンク対応版）
    
    Args:
        query (str): 検索クエリ文字列
        top_k (int, optional): 返却する上位結果数。デフォルトは5
        filtering (str, optional): ソース種別でフィルタリング。Noneの場合は全件を対象とする
    
    Returns:
        List[Dict[str, Any]]: 検索結果のリスト
    """
    # 1. クエリをベクトル化
    query_vector = embed_text(query)
    
    # 2. データベース接続
    connection = oracledb.connect(**db_params)
    cursor = connection.cursor()
    
    try:
        # 3. ベクトル類似度検索
        if filtering:
            sql = f"""
            SELECT 
                c.chunk_id,
                c.document_id,
                s.filename,
                c.chunk_text,
                VECTOR_DISTANCE(c.embedding, :query_vector, COSINE) as distance
            FROM {chunks_table} c
            JOIN {source_documents_table} s ON c.document_id = s.document_id
            WHERE s.filtering = :filtering
            ORDER BY VECTOR_DISTANCE(c.embedding, :query_vector, COSINE)
            FETCH FIRST :top_k ROWS ONLY
            """
            
            cursor.execute(sql, {
                'query_vector': query_vector,
                'filtering': filtering,
                'top_k': top_k
            })
        else:
            sql = f"""
            SELECT 
                c.chunk_id,
                c.document_id,
                s.filename,
                c.chunk_text,
                VECTOR_DISTANCE(c.embedding, :query_vector, COSINE) as distance
            FROM {chunks_table} c
            JOIN {source_documents_table} s ON c.document_id = s.document_id
            ORDER BY VECTOR_DISTANCE(c.embedding, :query_vector, COSINE)
            FETCH FIRST :top_k ROWS ONLY
            """
            
            cursor.execute(sql, {
                'query_vector': query_vector,
                'top_k': top_k
            })
        
        # 4. 結果を取得
        results = []
        for row in cursor:
            chunk_text_clob = row[3]
            chunk_text_content = chunk_text_clob.read() if chunk_text_clob is not None else ""
            
            results.append({
                'chunk_id': row[0],
                'document_id': row[1],
                'filename': row[2],
                'chunk_text': chunk_text_content,
                'distance': float(row[4])
            })
        
        return results
    finally:
        cursor.close()
        connection.close()


In [ ]:
####################################
# Rerankerモデルの初期化 (v2) #
####################################
print("\n" + "="*60)
print("Rerankerモデル (japanese-reranker-base-v2) を初期化中...")
print("="*60 + "\n")

def detect_device():
    """最適なデバイスを自動検出"""
    if torch.cuda.is_available():
        return "cuda"
    elif hasattr(torch, "mps") and torch.mps.is_available():
        return "mps"
    return "cpu"

device = detect_device()
print(f"✓ 検出デバイス: {device}")

print("✓ モデルをロード中...")
reranker_model = CrossEncoder(
    'hotchpotch/japanese-reranker-base-v2',
    max_length=512,
    device=device
)

if device in ["cuda", "mps"]:
    print("✓ half精度に変換中...")
    reranker_model.model.half()

print(f"✓ Rerankerモデルの初期化完了\n")

In [ ]:
##########
# Rerank #
##########

def rerank_chunks(query: str, chunks: List[Dict], top_n: int) -> List[Dict]:
    """
    hotchpotch/japanese-reranker-base-v2を使用してチャンクを再ランク付け
    
    Args:
        query (str): ユーザーのクエリ文字列
        chunks (List[Dict]): Vector Searchで取得したチャンクのリスト
        top_n (int): 最終的に返すチャンク数
    
    Returns:
        List[Dict]: Rerankされたチャンクのリスト（rerank_scoreを含む）
    """
    if not chunks:
        return []
    
    # チャンクテキストのリストを作成
    documents = [chunk['chunk_text'] for chunk in chunks]
    
    # クエリとドキュメントのペアを作成
    pairs = [[query, doc] for doc in documents]
    
    try:
        # Rerankerでスコア計算
        scores = reranker_model.predict(
            pairs, 
            show_progress_bar=False,
            batch_size=32
        )
        
        # 元のチャンクにrerankスコアを追加
        for chunk, score in zip(chunks, scores):
            chunk['rerank_score'] = float(score)
        
        # スコアでソート（降順）
        chunks.sort(key=lambda x: x['rerank_score'], reverse=True)
        
        # 上位top_n件を返す
        return chunks[:top_n]
        
    except Exception as e:
        print(f"⚠ Rerankエラー: {e}")
        print(f"⚠ Vector Searchの結果上位{top_n}件をそのまま返します")
        # Fallback: distanceでソート（小さい方が類似）
        return sorted(chunks, key=lambda x: x.get('distance', float('inf')))[:top_n]

In [ ]:
###########
# 回答生成 #
###########

def generate_answer(
    query: str,
    contexts: str,
    chat_model: str = "cohere.command-a-03-2025",
    max_tokens: int = 1000,
    temperature: float = 0.3,
    max_retries: int = 3,
    retry_delay: int = 60,
    answer_prompt: str = ""
):
    """
    検索結果を元に回答を生成（Cohere専用）
    
    Args:
        query (str): ユーザーの質問
        contexts (str): 参考ドキュメントのテキスト（結合済み）
        chat_model (str): 使用するモデルID。以下の2つが使えます。
            - "cohere.command-a-03-2025" (デフォルト)
            - "cohere.command-r-plus-08-2024"
        max_tokens (int): 最大トークン数
        temperature (float): 温度パラメータ（0-1、創造性の制御）
        max_retries (int): HTTP 429発生時の最大リトライ回数。デフォルトは3
        retry_delay (int): リトライ間隔の初期値（秒）。デフォルトは60秒
        answer_prompt (str): 回答生成時の指示文
    
    Returns:
        Dict[str, Any]: 生成結果の辞書。以下のキーを持つ:
            - answer (str): 生成された回答テキスト
            - model_used (str): 使用したモデルID
    """
    # プロンプト作成
    prompt = f"""以下のドキュメントを参考に、質問に回答してください。

【参考ドキュメント】
{contexts}

【質問】
{query}

【回答】
{answer_prompt}
"""
    
    # リクエスト作成
    chat_request = CohereChatRequest(
        message=prompt,
        max_tokens=max_tokens,
        temperature=temperature,
        frequency_penalty=0,
        top_p=0.75,
        top_k=0
    )
    
    chat_detail = ChatDetails(
        serving_mode=OnDemandServingMode(model_id=chat_model),
        compartment_id=compartment_id,
        chat_request=chat_request
    )
    
    # リトライロジック
    for attempt in range(max_retries + 1):
        try:
            # 回答生成実行
            response = generative_ai_inference_client.chat(chat_detail)
            
            # レスポンスから回答を取得
            answer = response.data.chat_response.text #type: ignore
            
            return {
                'answer': answer,
                'model_used': chat_model
            }
        
        except Exception as e:
            # HTTP 429エラー（Rate Limit）の場合
            if hasattr(e, 'status') and e.status == 429:
                if attempt < max_retries:
                    # 指数バックオフでリトライ間隔を延長
                    wait_time = retry_delay * (2 ** attempt)
                    print(f"HTTP 429エラー発生。{wait_time}秒後にリトライします... (試行 {attempt + 1}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    # 最大リトライ回数に達した場合
                    raise Exception(f"最大リトライ回数({max_retries})に達しました。HTTP 429エラーが継続しています。") from e
            else:
                # 429以外のエラーは即座に再送出
                raise

In [ ]:
################
# RAGAS評価関数 #
################

def evaluate_with_ragas(question, answer, contexts, ground_truth):
    """
    RAGシステムの出力をRAGASで評価
    
    Args:
        question (List[str]): 質問のリスト
        answer (List[str]): RAGシステムが生成した回答のリスト
        contexts (List[List[str]]): 各質問に対して検索されたコンテキストのリスト（リストのリスト）
        ground_truth (List[str]): 正解となる回答のリスト
    
    Returns:
        result (dict): 評価結果
            - faithfulness
            - answer_relevancy
            - context_precision
            - context_recall
    """
    # RAGAS評価専用のクライアント（タイムアウト延長版）
    generative_ai_client_for_ragas = oci.generative_ai_inference.GenerativeAiInferenceClient(
        config=config, 
        service_endpoint=service_endpoint, 
        retry_strategy=oci.retry.NoneRetryStrategy(), 
        timeout=(30, 600)  # 接続30秒、読み取り600秒に延長
    )
    
    # LLMの準備
    llm4eval = ChatOCIGenAI(
        model_id="cohere.command-a-03-2025",
        service_endpoint=service_endpoint,
        compartment_id=compartment_id,
        is_stream=False,
        model_kwargs={"temperature": 0.0, "max_tokens": 4000},
        auth_type="API_KEY",
        client=generative_ai_client_for_ragas)  # タイムアウト延長版クライアントを使用

    # 埋め込みモデルの準備
    embeddings4eval = OCIGenAIEmbeddings(
        model_id=embedding_model,
        service_endpoint=service_endpoint,
        compartment_id=compartment_id,
        truncate="END",
        auth_type="API_KEY",
        client=generative_ai_client_for_ragas)  # タイムアウト延長版クライアントを使用

    # データセットの作成
    ds = Dataset.from_dict(
        {
            "question": question,
            "answer": answer,
            "contexts": contexts,
            "ground_truth": ground_truth,
        }
    )

    # OCI Generative AI Cohere Chat用のfinished_parserを実装
    def my_finished_parser(response: LLMResult) -> bool:
        if (response.generations 
            and response.generations[0] 
            and response.generations[0][0].generation_info 
            and response.generations[0][0].generation_info.get('finish_reason') == 'COMPLETE'):
            return True
        return False

    # 評価用LLMのインスタンス生成（ここで、finished_parserを設定）
    evaluator_llm = LangchainLLMWrapper(llm4eval, is_finished_parser=my_finished_parser)

    # 評価用埋め込みモデルのインスタンス生成
    evaluator_embeddings = LangchainEmbeddingsWrapper(embeddings4eval)

    # メトリクスのインスタンスを作成
    metrics = [
        Faithfulness(llm=evaluator_llm),
        AnswerCorrectness(llm=evaluator_llm, embeddings=evaluator_embeddings),
        ContextPrecision(llm=evaluator_llm),
        ContextRecall(llm=evaluator_llm)
]

    # 評価の実行（エラー時は明示的に停止）
    result = evaluate(ds, metrics, raise_exceptions=True)

    return result

In [ ]:
###################################
# メイン処理1: Excelの質問への回答生成 #
###################################

# ベクトル検索のパラメータ
TOP_K = 20  # ベクトル検索の上位何件を返却するか。Rerank無効時はこの値がコンテキスト件数

# Rerankのパラメータ
RERANK_ENABLED = True # True or False
RERANK_TOP_N = 5  # Rerank後に上位何件をコンテキストとして利用するか

# 回答生成時のパラメータ
TEMPERATURE = 0.3
ANSWER_PROMPT = """
参考ドキュメントの情報に基づいて、正確に回答してください。ドキュメントに情報がない場合は、その旨を伝えてください。
参考ドキュメントの情報を回答に利用した場合は、参考ドキュメントのファイル名も提示してください。
"""

# FAQ Excelファイルからデータロード
faq_df = load_excel_from_object_storage(config, bucket_name, object_name)

print(f"FAQデータに対してRAG処理を開始します（全{len(faq_df)}件）\n")

# answer, contexts, パフォーマンス計測用の列を新規作成
faq_df['answer'] = None
faq_df['contexts'] = None
faq_df['vector_search_time'] = 0.0
faq_df['rerank_time'] = 0.0
faq_df['generation_time'] = 0.0
faq_df['total_time'] = 0.0

# 全体処理時間の計測開始
overall_start_time = time.time()

# 各質問に対して処理
for idx, row in faq_df.iterrows():
    question = row['question']
    filter_value = row['filter']
    
    if pd.notna(filter_value) and filter_value != "":
        filtering = filter_value
    else:
        filtering = None
    
    print(f"[{idx + 1}/{len(faq_df)}] 処理中: {question[:50]}...")
    
    try:
        # 1. ベクトル検索でコンテキストを取得（時間計測）
        vector_search_start = time.time()
        candidates = vector_search(
            query=question,
            top_k=TOP_K,
            filtering=filtering
        )
        vector_search_time = time.time() - vector_search_start
        faq_df.at[idx, 'vector_search_time'] = vector_search_time

        # 2. Rerankingで絞り込み（時間計測）
        rerank_start = time.time()
        if RERANK_ENABLED:
            search_results = rerank_chunks(question, candidates, top_n=RERANK_TOP_N)
        else:
            search_results = candidates[:RERANK_TOP_N]
        rerank_time = time.time() - rerank_start
        faq_df.at[idx, 'rerank_time'] = rerank_time
 
        # 3. contextsを作成（LLMに渡す用 & DataFrame格納用で同一）
        contexts = "\n\n".join([
            f"[ドキュメント {i+1}: {result['filename']}]\n{result['chunk_text']}"
            for i, result in enumerate(search_results)
        ])
        
        # 4. LLMで回答生成（時間計測）
        generation_start = time.time()
        result = generate_answer(
            query=question,
            contexts=contexts,
            chat_model=chat_model,
            temperature=TEMPERATURE,
            answer_prompt=ANSWER_PROMPT
        )
        generation_time = time.time() - generation_start
        faq_df.at[idx, 'generation_time'] = generation_time
        
        # 5. 合計時間を計算
        total_time = vector_search_time + rerank_time + generation_time
        faq_df.at[idx, 'total_time'] = total_time
        
        # 6. DataFrameに結果を格納
        faq_df.at[idx, 'answer'] = result['answer']
        faq_df.at[idx, 'contexts'] = contexts
        
        print(f"  ✓ 完了 (検索結果: {len(search_results)}件, 処理時間: {total_time:.2f}秒)")
        print(f"    - ベクトル検索: {vector_search_time:.2f}秒")
        print(f"    - Rerank: {rerank_time:.2f}秒")
        print(f"    - 回答生成: {generation_time:.2f}秒")
        
    except Exception as e:
        print(f"  ✗ エラー: {e}")
        faq_df.at[idx, 'answer'] = ""
        faq_df.at[idx, 'contexts'] = ""
        continue

# 全体処理時間の計測終了
overall_end_time = time.time()
overall_processing_time = overall_end_time - overall_start_time

print(f"\n{'='*60}")
print(f"✓ RAG処理が完了しました")
print(f"{'='*60}")
print(f"全体処理時間: {overall_processing_time:.2f}秒")
print(f"平均処理時間: {overall_processing_time/len(faq_df):.2f}秒/件")
print(f"\n【処理時間の統計】")
print(f"  ベクトル検索平均: {faq_df['vector_search_time'].mean():.2f}秒")
print(f"  Rerank平均: {faq_df['rerank_time'].mean():.2f}秒")
print(f"  回答生成平均: {faq_df['generation_time'].mean():.2f}秒")
print(f"  合計平均: {faq_df['total_time'].mean():.2f}秒")

# 結果を確認
print("\n処理結果:")
print(faq_df[['id', 'question', 'ground_truth', 'filter', 'answer', 'total_time']].head())

# メタデータを作成（パフォーマンス統計を含む）
metadata = {
    'パラメータ': [
        'TOP_K (ベクトル検索件数)',
        'RERANK_ENABLED (Rerankが有効か)',
        'RERANK_TOP_N (Rerank後件数)',
        'TEMPERATURE (温度)',
        'embedding_model',
        'chat_model',
        'rerank_model',
        'service_endpoint',
        '実行日時',
        'FAQ件数',
        '全体処理時間（秒）',
        '平均処理時間/件（秒）',
        'ベクトル検索平均時間（秒）',
        'Rerank平均時間（秒）',
        '回答生成平均時間（秒）'
    ],
    '設定値': [
        TOP_K,
        RERANK_ENABLED,
        RERANK_TOP_N,
        TEMPERATURE,
        embedding_model,
        chat_model,
        rerank_model,
        service_endpoint,
        datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        len(faq_df),
        f"{overall_processing_time:.2f}",
        f"{overall_processing_time/len(faq_df):.2f}",
        f"{faq_df['vector_search_time'].mean():.2f}",
        f"{faq_df['rerank_time'].mean():.2f}",
        f"{faq_df['generation_time'].mean():.2f}"
    ]
}
metadata_df = pd.DataFrame(metadata)

# ファイル名生成（タイムスタンプのみ）
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_filename = f"rag_result_{timestamp}.xlsx"

# Object Storageにアップロード（メタデータ付き）
save_to_object_storage(faq_df, config, bucket_name, output_filename, metadata_df=metadata_df)

In [ ]:
#############################
# メイン処理2: RAGAS評価の実行 #
#############################

print(f"\n{'='*60}")
print(f"RAGAS評価処理を開始します")
print(f"{'='*60}\n")

# 評価対象のファイル名を指定
input_filename = output_filename  # 個別で対象ファイルを指定する場合は、ここを修正してください

# 1. Object StorageからExcelファイルをダウンロード
print(f"1. Object Storageからファイルをダウンロード中...")
df_for_ragas = load_excel_from_object_storage(config, bucket_name, input_filename, sheet_name='Results')

print(f"   ✓ ダウンロード完了: {len(df_for_ragas)}件のデータ")

# 2. RAGAS評価用のデータを準備
print(f"\n2. RAGAS評価用データを準備中...")

# DataFrameから必要なデータを抽出
questions = df_for_ragas['question'].tolist()
answers = df_for_ragas['answer'].tolist()
ground_truths = df_for_ragas['ground_truth'].tolist()

# contextsをリストのリストに変換（RAGASの要求形式）
# 各ドキュメントを個別要素に分割する
contexts_list = []
for ctx in df_for_ragas['contexts'].tolist():
    if pd.isna(ctx) or ctx == "":
        contexts_list.append([""])
    else:
        # "[ドキュメント" で始まる行でドキュメントを分割
        lines = ctx.split('\n')
        individual_docs = []
        current_doc = ""
        
        for line in lines:
            if line.startswith('[ドキュメント'):
                # 新しいドキュメントが始まる
                if current_doc:
                    individual_docs.append(current_doc.strip())
                current_doc = line + '\n'
            else:
                current_doc += line + '\n'
        
        # 最後のドキュメントを追加
        if current_doc:
            individual_docs.append(current_doc.strip())
        
        if individual_docs:
            contexts_list.append(individual_docs)
        else:
            # 分割できなかった場合は元のまま
            contexts_list.append([ctx])

print(f"   ✓ データ準備完了")

# 3. RAGAS評価を実行
print(f"\n3. RAGAS評価を実行中...")
print(f"   （この処理には数分かかる場合があります）")

try:
    ragas_result = evaluate_with_ragas(questions, answers, contexts_list, ground_truths)
    
    print(f"\n   ✓ RAGAS評価完了")
    
except Exception as e:
    print(f"\n   ✗ RAGAS評価エラー: {e}")
    raise

# 4. 評価結果をDataFrameに追記
print(f"\n4. 評価結果をDataFrameに追記中...")

result_df = ragas_result.to_pandas() #type: ignore

# 各メトリクスをDataFrameに追加
df_for_ragas['faithfulness'] = result_df['faithfulness']
df_for_ragas['answer_correctness'] = result_df['answer_correctness']
df_for_ragas['context_precision'] = result_df['context_precision']
df_for_ragas['context_recall'] = result_df['context_recall']

print(f"   ✓ 追記完了")

# 5. メタデータシートも読み込んで保持
print(f"\n5. メタデータを読み込み中...")
metadata_df = load_excel_from_object_storage(config, bucket_name, input_filename, sheet_name='Settings')
print(f"   ✓ メタデータ読み込み完了")

# 6. 更新したExcelをObject Storageにアップロード
print(f"\n6. 評価結果をObject Storageにアップロード中...")

# ファイル名に _ragas サフィックスを追加
output_filename = input_filename.replace('.xlsx', '_ragas.xlsx')

# アップロード（メタデータも一緒に保存）
save_to_object_storage(df_for_ragas, config, bucket_name, output_filename, metadata_df=metadata_df)
print(f"   ✓ アップロード完了")

print(f"\n{'='*60}")
print(f"RAGAS評価処理が完了しました")
print(f"{'='*60}\n")

# 評価結果の確認
print("\n評価結果サンプル:")
print(df_for_ragas[['id', 'question', 'filter', 'faithfulness', 'answer_correctness', 'context_precision', 'context_recall']].head())